# Model Inference Demo

## Step 0: Import required modules

In [1]:
import torch
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
import os

import caffe2.python.onnx.backend as backend

import onnx

from PIL import Image

## Step 1: Define transformations to be applied to images

In [4]:
device = torch.device('cpu')
#torch.load('my_file.pt', map_location=lambda storage, loc: storage)
model = torch.load('chest_xray_kaggle.pt', map_location=lambda storage, loc: storage)
dummy_input = torch.randn(1, 3, 224, 224)
#model = model.cpu().double()

torch.onnx.export(model, dummy_input, "chest_xray_kaggle.onnx")

AttributeError: 'LogSoftmax' object has no attribute '_state_dict_hooks'

In [ ]:
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

## Step 3: Load the model

In [ ]:
model = torch.load("caltech_10_model_8.pt", map_location='cpu')
# In case of a GPU machine, use:
# model = torch.load("chest_xray_kaggle_model_23_good.pt")

## Step 4: Export the model in ONNX format

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, dummy_input, "caltech_10_model.onnx")

## Step 5: Model Inference

In [ ]:
# First load the onnx model
model = onnx.load("caltech_10_model.onnx")

In [ ]:
# Prepare the backend
rep = backend.prepare(model, device="CPU")

In [ ]:
# Transform the image
transform = image_transforms['test']

test_image_name = "giraffe-1330814_640.jpg"

test_image = Image.open(test_image_name)

plt.imshow(test_image)

test_image_tensor = transform(test_image)

if torch.cuda.is_available():
    test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
else:
    test_image_tensor = test_image_tensor.view(1, 3, 224, 224)

In [ ]:
# Convert the tensor to numpy array
np_image = test_image_tensor.numpy()
outputs = rep.run(np_image.astype(np.float32))

In [ ]:
# Dictionary with class name and index
idx_to_class = {0: 'bear', 1: 'chimp', 2: 'giraffe', 3: 'gorilla', 4: 'llama', 5: 'ostrich', 6: 'porcupine', 7: 'skunk', 8: 'triceratops', 9: 'zebra'}

ps = torch.exp(torch.from_numpy(outputs[0]))
topk, topclass = ps.topk(3, dim=1)
for i in range(3):
    print("Prediction", i+1, ":", idx_to_class[topclass.cpu().numpy()[0][i]], ", Score: ", topk.cpu().detach().numpy()[0][i])
    #print("Predcition", i+1, ":", topclass[0][i], ", Score: ", topk[0][i])